<a href="https://colab.research.google.com/github/TBoedt/ThesisR-T/blob/main/Sentiment_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers[sentencepiece]
!pip install Xformers
!pip install transformers
!pip install vaderSentiment


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.8 MB/s eta 0:00:00


In [ ]:
###mount bucket
#auth
from google.colab import auth
auth.authenticate_user()

#install gcsfuse
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# Connect Colab to GCS Bucket
!mkdir mounted_bucket
!gcsfuse --implicit-dirs scraped_articles mounted_bucket

#check files
!ls mounted_bucket

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  88633      0 --:--:-- --:--:-- --:--:-- 88633
OK
33 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 14.0 MB of archives.
After this operation, 31.2 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 122532 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.42.5_amd64.deb ...
Unpacking gcsfuse (0.42.5) ...
Setting up gcsfuse (0.42.5) ...
I0524 08:47:47.267510 2023/05/24 08:47:47.267473 Start gcsfuse/0.42.5 (Go version go1.20.3) for app "" using mount point: /content/mounted_bucket
 Climate_Labels_Dataset.parquet
 Final_Data
 Final_scrape_Reane
 Lexicons
 pyabsa_sc

In [ ]:
# # get data (you can do this here or just rever to /mouted_bucket/Wash.... whenever you need it. This just puts it on the virtual machine alreadys)
# !gsutil cp /content/mounted_bucket/Washington_Post_2016_Articles .
# !gsutil cp /content/mounted_bucket/Washington_Post_2017_Articles .
# !gsutil cp /content/mounted_bucket/Washington_Post_2018_Articles .
# !gsutil cp /content/mounted_bucket/Washington_Post_2019_Articles .
# !gsutil cp /content/mounted_bucket/Washington_Post_2020_Articles .

# !gsutil cp /content/mounted_bucket/Wall_Street_Journal_2016_Articles .
# !gsutil cp /content/mounted_bucket/Wall_Street_Journal_2017_Articles .
# !gsutil cp /content/mounted_bucket/Wall_Street_Journal_2018_Articles .
# !gsutil cp /content/mounted_bucket/Wall_Street_Journal_2019_Articles .
# !gsutil cp /content/mounted_bucket/Wall_Street_Journal_2020_Articles .

!gsutil cp /content/mounted_bucket/Final_Data/WP_Final_2016.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WP_Final_2017.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WP_Final_2018.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WP_Final_2019.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WP_Final_2020.parquet .

!gsutil cp /content/mounted_bucket/Final_Data/WSJ_Final_2016.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WSJ_Final_2017.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WSJ_Final_2018.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WSJ_Final_2019.parquet .
!gsutil cp /content/mounted_bucket/Final_Data/WSJ_Final_2020.parquet .

# !gsutil cp /content/mounted_bucket/Climate_Labels_Dataset.parquet .
!gsutil cp /content/mounted_bucket/Lexicons/EPA_BBC_Lexicon .

Copying file:///content/mounted_bucket/Final_Data/WSJ_Final_2016.parquet...
-
Operation completed over 1 objects/149.2 MiB.                                    
Copying file:///content/mounted_bucket/Final_Data/WSJ_Final_2017.parquet...
-
Operation completed over 1 objects/73.7 MiB.                                     
Copying file:///content/mounted_bucket/Final_Data/WSJ_Final_2018.parquet...
-
Operation completed over 1 objects/99.0 MiB.                                     
Copying file:///content/mounted_bucket/Final_Data/WSJ_Final_2019.parquet...
-
Operation completed over 1 objects/97.3 MiB.                                     
Copying file:///content/mounted_bucket/Final_Data/WSJ_Final_2020.parquet...
-
Operation completed over 1 objects/105.6 MiB.                                    
Copying file:///content/mounted_bucket/Lexicons/EPA_BBC_Lexicon...
/ [1 files][  4.2 KiB/  4.2 KiB]                                                
Operation completed over 1 objects/4.2 KiB.         

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM,  AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
import pyarrow.parquet as pq
import pyarrow as pyarrow
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import precision_score, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from transformers import pipeline


In [ ]:
def summary(df_with_text, name, model_name, max_lenght_input=-1):
    data_in_list = df_with_text[name].tolist()
    tokenizer_sum = AutoTokenizer.from_pretrained(model_name)
    model_sum = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    summarizer = pipeline('summarization', model=model_sum, tokenizer = tokenizer_sum) 

    if max_lenght_input>=0:
      df_with_text['summary'] = summarizer(data_in_list, max_length=max_lenght_input)

    else:
      df_with_text['summary'] = summarizer(data_in_list)

def classification(df_with_text, name, model_name, max_lenght_input=-1):
    data_in_list = df_with_text[name].tolist()
    tokenizer_clas = AutoTokenizer.from_pretrained(model_name)
    model_clas = AutoModelForSequenceClassification.from_pretrained(model_name)
    classification = pipeline('text-classification', model=model_clas, tokenizer = tokenizer_clas) 

    if max_lenght_input>=0:
      df_with_text[model_name] = classification(data_in_list, max_length=max_lenght_input, truncation=True)

    else:
      df_with_text[model_name] = classification(data_in_list)

def calculate_sentiment_score(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

def vader_sentiment(df_with_text, name):
    df_with_text['sentiment_score_vader'] = df_with_text[name].progress_apply(calculate_sentiment_score)

def extract_sentences_with_lexicon(text, lexicon):
    sentences = text.split('.')
    lexicon_set = set(lexicon)
    result = []
    for sentence in sentences:
        if any(term in sentence.lower() for term in lexicon_set):
            result.append(sentence)
    return result


def get_average_vader_sentiment_score(sentences):
  analyzer = SentimentIntensityAnalyzer()
  scores = [analyzer.polarity_scores(sentence)['compound'] for sentence in sentences]
  if(len(scores)>0):
    return sum(scores) / len(scores)
  else:
    return 'NaN'

def remove_newlines(text):
  return text.replace('\n', '')

def categorize_score(score, negative_threshold, positive_threshold):
  if score < negative_threshold:
      return -1
  elif score > positive_threshold:
      return 1
  else:
      return 0

def extract_sentences_with_lexicon(text, lexicon):
    sentences = text.split('.')
    lexicon_set = set(lexicon)
    result = []
    for sentence in sentences:
        if any(term in sentence.lower() for term in lexicon_set):
            result.append(sentence)
    return result

In [ ]:
version='v2'
year = '5'
name= 'WSJ_'+year
# lexicon_name = 'EPA_Lexicon'

# lexicon = pd.read_csv(lexicon_name)
# lexicon = lexicon['Lexicon'].to_list()
df = pd.read_parquet(name)
df
# df_with_text = df[df['Climate']=='Yes'].reset_index(drop=True)


,Title,Text,Link,Date,News Paper
400000,Tax Package Presents Mixed Bag For Retirement ...,Advertisement - Scroll to Continue NEWSLE...,https://www.wsj.com/articles/tax-package-prese...,19/12/2017,Wall_Street_Journal
400001,Big Banks Steal Smaller Brokers’ Thunder,It’s a good time to be a megabank. In many bu...,https://www.wsj.com/articles/midsize-banks-got...,07/05/2018,Wall_Street_Journal
400002,Greenlight Raising $10 Million Series A to Pro...,Greenlight Financial Technology Inc. has rais...,https://www.wsj.com/articles/greenlight-raisin...,19/12/2017,Wall_Street_Journal
400003,Red Carpet at Golden Globe Awards 2016,,http://www.wsj.com/articles/red-carpet-at-gold...,10/01/2016,Wall_Street_Journal
400004,5AM Ventures Adds Amy Burroughs as EIR,5AM Ventures has hired Amy Burroughs as an ex...,https://www.wsj.com/articles/5am-ventures-adds...,19/12/2017,Wall_Street_Journal
...,...,...,...,...,...
413499,The 10-Point.,"Good evening from Davos, Switzerland, on the ...",https://www.wsj.com/articles/the-10-point-1516...,26/01/2018,Wall_Street_Journal
413500,Mystical Musical Mélange,"The composer, conductor and edu...",https://www.wsj.com/articles/mystical-musical-...,26/01/2018,Wall_Street_Journal
413501,Colgate’s Checkup Doesn’t Go Well,A disappointing fourth quarter at Colgate-Pal...,https://www.wsj.com/articles/colgates-checkup-...,26/01/2018,Wall_Street_Journal
413502,"Blue Water Energy, Blackstone to Invest Up to ...",London investment firm Blue Water Energy and ...,https://www.wsj.com/articles/blue-water-energy...,26/01/2018,Wall_Street_Journal


## Vader sentiment


In [ ]:
#### get data
for year in range(2016,2017):
  name= 'WP_Final_'+str(year)+'.parquet'
  df = pd.read_parquet(name)
  df_with_text = df[df['Climate']=='Yes'].reset_index(drop=True)
  # df['Text'] = df['Text'].str.replace('\n', ' ')
  #### Vader sentiment

  #load TQDM for progress bar 
  tqdm.pandas()

  #input 
  # df_with_text['sentences_with_lexicon'] = df_with_text['Text'].progress_apply(lambda x: extract_sentences_with_lexicon(x, lexicon))

  # #select only relevant rows and columns
  # df_with_text= df_with_text[(df_with_text['Final_Climate_Change_Level_Label']=='High') | (df_with_text['Final_Climate_Change_Level_Label']=='Medium')].reset_index(drop=True)
  # df_with_text =df_with_text.drop(['Sentiment_Label', 'Sentiment_Label_R', 'Level_Climate_Change_Topic', 'Level_Climate_Change_Topic_R', 'was_I_retarded?'], axis=1)


  #vader sentiment of the whole text
  vader_sentiment(df_with_text, 'Text')


  # #get the vader score for the relevant scentences and get the value
  # df_with_text['Vader_lexicon_scentences_sentiment'] = df_with_text['sentences_with_lexicon'].progress_apply(lambda x: get_average_vader_sentiment_score(x))
  # # df_with_text['Vader_lexicon_scentences_sentiment'] = df_with_text['Vader_lexicon_scentences_sentiment'].apply(lambda x: categorize_score(x, -.03, +.03))
  # df_with_text



  # # Vader per word
  # sia = SentimentIntensityAnalyzer()
  # df_with_text['words'] = df_with_text['Text'].str.split()
  # df_with_text['positive_words'] = df_with_text['words'].progress_apply(lambda x: sum(sia.polarity_scores(word)['compound'] > 0 for word in x))
  # df_with_text['negative_words'] = df_with_text['words'].progress_apply(lambda x: sum(sia.polarity_scores(word)['compound'] < 0 for word in x))
  # df_with_text['total_words'] = df_with_text['words'].progress_apply(len)
  # df_with_text['Vader_total_sentiments_based_on_words'] = (df_with_text['positive_words']-df_with_text['negative_words'])/(df_with_text['negative_words']+df_with_text['positive_words'])
  # # df_with_text['Vader_total_sentiments_based_on_words'] = df_with_text['total_sentiments_based_on_words'].apply(lambda x: categorize_score(x, +.1, .2))
  # df_with_text = df_with_text.drop(['words', 'positive_words', 'negative_words', 'total_words'], axis = 1)

  # df_with_vader = df_with_text


  #### ML sentiment 
  classification(df_with_text, 'Text',"climatebert/distilroberta-base-climate-sentiment",512)
  replace_dict_CB = {'neutral': 0, 'opportunity': 1, 'risk': -1}
  df_with_text['climatebert/distilroberta-base-climate-sentiment'] = df_with_text['climatebert/distilroberta-base-climate-sentiment'].apply(lambda x: x['label']).replace(replace_dict_CB)

  # classification(df_with_text, 'Text','mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis',512)
  # replace_dict_FN = {'neutral': 0, 'positive': 1, 'negative': -1}
  # df_with_text['mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']= df_with_text['mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'].apply(lambda x: x['label']).replace(replace_dict_FN)

  ##### Save
  df_with_text.to_parquet(f'mounted_bucket/Final_Data/sentiment_files/{name}_sentiment_{version}')

100%|██████████| 722/722 [04:00<00:00,  3.01it/s]


In [ ]:
df_with_text.to_parquet(f'mounted_bucket/Final_Data/sentiment_files/{name}_sentiment_{version}')

## ML sentiment

In [ ]:
#sentiment using ML
# classification(df_with_text, 'Text',"Seethal/sentiment_analysis_generic_dataset",512)
classification(df_with_text, 'Text',"climatebert/distilroberta-base-climate-sentiment",512)
replace_dict_CB = {'neutral': 0, 'opportunity': 1, 'risk': -1}
df_with_text['climatebert/distilroberta-base-climate-sentiment'] = df_with_text['climatebert/distilroberta-base-climate-sentiment'].apply(lambda x: x['label']).replace(replace_dict_CB)

classification(df_with_text, 'Text','mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis',512)
replace_dict_FN = {'neutral': 0, 'positive': 1, 'negative': -1}
df_with_text['mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']= df_with_text['mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'].apply(lambda x: x['label']).replace(replace_dict_FN)

# classification(df_with_text, 'Text',"mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",512)
# df_with_text['mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']=.apply(lambda x: x['label'])
# neutral positive negative

# Sentiment based upon the word sentiments



In [ ]:
df_with_text=df_with_text.replace('NaN', 0)
df_with_text.to_parquet(f'mounted_bucket/Final_Data/sentiment_files/{name}_{lexicon_name}_sentiment_raw_{version}')

# df_with_text = df_with_text.drop(['sentences_with_lexicon'], axis=1)



In [ ]:
import nltk
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

df_with_text = pd.read_table('Climate_Labels_Dataset.parquet').to_pandas()
#select only relevant rows and columns
df_with_text= df_with_text[(df_with_text['Final_Climate_Change_Level_Label']=='High') | (df_with_text['Final_Climate_Change_Level_Label']=='Medium')].reset_index(drop=True)
df_with_text =df_with_text.drop(['Sentiment_Label', 'Sentiment_Label_R', 'Level_Climate_Change_Topic', 'Level_Climate_Change_Topic_R', 'was_I_retarded?'], axis=1)


# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Split the text into words and calculate sentiment
df_with_text['words'] = df_with_text['Text'].str.split()
df_with_text['positive_words'] = df_with_text['words'].apply(lambda x: sum(sia.polarity_scores(word)['compound'] > 0 for word in x))
df_with_text['negative_words'] = df_with_text['words'].apply(lambda x: sum(sia.polarity_scores(word)['compound'] < 0 for word in x))
df_with_text['total_words'] = df_with_text['words'].apply(len)
df_with_text['total_sentiments_based_on_words'] = (df_with_text['positive_words']-df_with_text['negative_words'])/(df_with_text['negative_words']+df_with_text['positive_words'])
df_with_text['total_sentiments_based_on_words'] = df_with_text['total_sentiments_based_on_words'].apply(lambda x: categorize_score(x, +.1, .2))


# Display the updated DataFrame
print('\n')
sent_crosstab(df_with_text, 'Final_Sentiment_Label', 'total_sentiments_based_on_words')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


FileNotFoundError: ignored

# Checking

In [ ]:
def sent_crosstab(df, truth, guess):
    # create a crosstab of the two labels
    crosstab = pd.crosstab(df[truth], df[guess])

    print(crosstab)
    # calculate precision and accuracy for each value
    for col in crosstab.columns:
        precision = precision_score(df[truth], df[guess], labels=[col], average='macro')
        accuracy = accuracy_score(df[truth], df[guess])
        print(f"Precision for {col}: {precision}")
        print(f"Accuracy for {col}: {accuracy}")

In [ ]:
sent_crosstab(df_with_text, 'Final_Sentiment_Label', 'sentiment_score_ml')

print('\n')
sent_crosstab(df_with_text, 'Final_Sentiment_Label', 'sentiment_score_vader')

print('\n')
sent_crosstab(df_with_text, 'Final_Sentiment_Label', 'scentences_vader_score')

# Rounding


In [ ]:
# df_with_text['Vader_lexicon_scentences_sentiment'] = df_with_text['Vader_lexicon_scentences_sentiment'].apply(lambda x: categorize_score(x, -.03, +.03))


## Testing

In [ ]:
# get data
version='v2'
year = '2017'
name= 'Climate_Labels_Dataset.parquet'
lexicon_name = 'EPA_BBC_Lexicon'

lexicon = pd.read_csv(lexicon_name)
lexicon = lexicon['Lexicon'].to_list()
df = pd.read_parquet(name)
df_with_text = df[df['Target']=='Yes'].reset_index(drop=True)
df_with_text

,Text,Link,Final_Climate_Change_Level_Label,Final_Sentiment_Label,Target
0,"On a Train trip north toward Aberdeen, the Sc...",https://www.wsj.com/articles/surfacing-review-...,Medium,-1,Yes
1,"During a visit to Detroit last year, Presiden...",https://www.wsj.com/articles/make-cars-great-a...,High,1,Yes
2,Investors holding more than $5 billion in Exx...,http://www.wsj.com/articles/calpers-pushes-exx...,High,1,Yes
3,As the grotesquerie masquerading as a presiden...,https://www.washingtonpost.com/opinions/what-i...,Medium,-1,Yes
4,The country’s natural-gas boom is putting muc...,https://www.wsj.com/articles/natural-gas-boom-...,High,-1,Yes
...,...,...,...,...,...
90,Apple’s recent efforts to trumpet its green in...,https://www.washingtonpost.com/news/energy-env...,High,1,Yes
91,President-elect Joe Biden said Thursday he pl...,https://www.wsj.com/articles/biden-picks-north...,High,1,Yes
92,WASHINGTON—President-elect Joe Biden is neari...,https://www.wsj.com/articles/biden-closes-in-o...,High,0,Yes
93,President Trump’s recent blowup over General ...,https://www.wsj.com/articles/the-electric-kool...,Medium,-1,Yes


In [ ]:
 #### Vader sentiment

#load TQDM for progress bar 
tqdm.pandas()

#input 
df_with_text['sentences_with_lexicon'] = df_with_text['Text'].progress_apply(lambda x: extract_sentences_with_lexicon(x, lexicon))

# #select only relevant rows and columns
# df_with_text= df_with_text[(df_with_text['Final_Climate_Change_Level_Label']=='High') | (df_with_text['Final_Climate_Change_Level_Label']=='Medium')].reset_index(drop=True)
# df_with_text =df_with_text.drop(['Sentiment_Label', 'Sentiment_Label_R', 'Level_Climate_Change_Topic', 'Level_Climate_Change_Topic_R', 'was_I_retarded?'], axis=1)


#vader sentiment of the whole text
vader_sentiment(df_with_text, 'Text')


#get the vader score for the relevant scentences and get the value
df_with_text['Vader_lexicon_scentences_sentiment'] = df_with_text['sentences_with_lexicon'].progress_apply(lambda x: get_average_vader_sentiment_score(x))
# df_with_text['Vader_lexicon_scentences_sentiment'] = df_with_text['Vader_lexicon_scentences_sentiment'].apply(lambda x: categorize_score(x, -.03, +.03))
df_with_text



# Vader per word
sia = SentimentIntensityAnalyzer()
df_with_text['words'] = df_with_text['Text'].str.split()
df_with_text['positive_words'] = df_with_text['words'].progress_apply(lambda x: sum(sia.polarity_scores(word)['compound'] > 0 for word in x))
df_with_text['negative_words'] = df_with_text['words'].progress_apply(lambda x: sum(sia.polarity_scores(word)['compound'] < 0 for word in x))
df_with_text['total_words'] = df_with_text['words'].progress_apply(len)
df_with_text['Vader_total_sentiments_based_on_words'] = (df_with_text['positive_words']-df_with_text['negative_words'])/(df_with_text['negative_words']+df_with_text['positive_words'])
# df_with_text['Vader_total_sentiments_based_on_words'] = df_with_text['total_sentiments_based_on_words'].apply(lambda x: categorize_score(x, +.1, .2))
df_with_text = df_with_text.drop(['words', 'positive_words', 'negative_words', 'total_words'], axis = 1)

df_with_vader = df_with_text


#### ML sentiment 
classification(df_with_text, 'Text',"climatebert/distilroberta-base-climate-sentiment",512)
replace_dict_CB = {'neutral': 0, 'opportunity': 1, 'risk': -1}
df_with_text['climatebert/distilroberta-base-climate-sentiment'] = df_with_text['climatebert/distilroberta-base-climate-sentiment'].apply(lambda x: x['label']).replace(replace_dict_CB)

classification(df_with_text, 'Text','mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis',512)
replace_dict_FN = {'neutral': 0, 'positive': 1, 'negative': -1}
df_with_text['mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']= df_with_text['mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'].apply(lambda x: x['label']).replace(replace_dict_FN)

##### Save
# df_with_text=df_with_text.replace('NaN', 0)
# df_with_text.to_parquet(f'mounted_bucket/Final_Data/sentiment_files/{name}_{lexicon_name}_sentiment_raw_{version}')

100%|██████████| 95/95 [00:00<00:00, 60889.19it/s]


In [ ]:
def sent_crosstab(df, truth, guess):
    # create a crosstab of the two labels
    crosstab = pd.crosstab(df[truth], df[guess])

    print(crosstab)
    # calculate precision and accuracy for each value
    for col in crosstab.columns:
        precision = precision_score(df[truth], df[guess], labels=[col], average='macro')
        accuracy = accuracy_score(df[truth], df[guess])
        print(f"Precision for {col}: {precision}")
        print(f"Accuracy for {col}: {accuracy}")

In [ ]:
df_with_text
# List of columns to update
columns_to_update = ['Final_Sentiment_Label',
                     'sentiment_score_vader',
                     'Vader_lexicon_scentences_sentiment',
                     'Vader_total_sentiments_based_on_words',
                     'climatebert/distilroberta-base-climate-sentiment',
                     'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']

# Update values in the specified columns
df_with_text[columns_to_update] = df_with_text[columns_to_update].applymap(
    lambda x: 'negative' if x < 0 else 'not negative'
)
# df_with_text[columns_to_update].dtypes

In [ ]:
sent_crosstab(df_with_text, 'Final_Sentiment_Label', 'climatebert/distilroberta-base-climate-sentiment'	)

climatebert/distilroberta-base-climate-sentiment  -1   0   1
Final_Sentiment_Label                                       
-1                                                34  18  10
 0                                                 3   6   2
 1                                                 0  13   9
Precision for -1: 0.918918918918919
Accuracy for -1: 0.5157894736842106
Precision for 0: 0.16216216216216217
Accuracy for 0: 0.5157894736842106
Precision for 1: 0.42857142857142855
Accuracy for 1: 0.5157894736842106


In [ ]:
to_check = ['sentiment_score_vader',
            'Vader_lexicon_scentences_sentiment',
            'Vader_total_sentiments_based_on_words',
            'climatebert/distilroberta-base-climate-sentiment',
            'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']

for name in to_check: 
    sent_crosstab(df_with_text, 'Final_Sentiment_Label',name)
    print('\n')

sentiment_score_vader  negative  not negative
Final_Sentiment_Label                        
negative                     21            41
not negative                  3            30
Precision for negative: 0.875
Accuracy for negative: 0.5368421052631579
Precision for not negative: 0.4225352112676056
Accuracy for not negative: 0.5368421052631579


Vader_lexicon_scentences_sentiment  negative  not negative
Final_Sentiment_Label                                     
negative                                  24            38
not negative                               6            27
Precision for negative: 0.8
Accuracy for negative: 0.5368421052631579
Precision for not negative: 0.4153846153846154
Accuracy for not negative: 0.5368421052631579


Vader_total_sentiments_based_on_words  negative  not negative
Final_Sentiment_Label                                        
negative                                     14            48
not negative                                  3            30


In [ ]:
to_check = ['sentiment_score_vader',
            'Vader_lexicon_scentences_sentiment',
            'Vader_total_sentiments_based_on_words',
            'climatebert/distilroberta-base-climate-sentiment',
            'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']

Accuracy = []
Precision = []
Recall = []
F1_score = []
Name = []
Treshhold = []
Technique = []
df = pd.DataFrame()
for name in to_check:
    cross_table = pd.crosstab(df_with_text['Final_Sentiment_Label'], df_with_text[name], margins=True)

    # calculate classification metrics using scikit-learn
    accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
    precision = cross_table.iloc[1,1] / (cross_table.iloc[0,1] + cross_table.iloc[1,1]) if cross_table.shape == (3,3) else 0
    recall = cross_table.iloc[1,1] / (cross_table.iloc[1,0] + cross_table.iloc[1,1]) if cross_table.shape == (3,3) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    Accuracy.append(accuracy)
    Precision.append(precision)
    Recall.append(recall)
    F1_score.append(f1_score)
    Name.append(name)
    Technique.append("Absolute Presences")
    
    df.append({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score},ignore_index=True)
df

<ipython-input-43-df6e4f047d92>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score},ignore_index=True)
<ipython-input-43-df6e4f047d92>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score},ignore_index=True)
<ipython-input-43-df6e4f047d92>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append({"Lexicon" : Name, "Technique": Technique, "Treshhold" : Treshhold, "Accuracy" : Accuracy, "Precision" 

""


In [ ]:
def get_metrics_df_3(df_text, names):    
  Accuracy = []
  Precision = []
  Recall = []
  F1_score = []
  Name = []
    
  for name in to_check:
    cross_table = pd.crosstab(df_with_text['Final_Sentiment_Label'], df_with_text[name], margins=True)


    # calculate classification metrics using scikit-learn
    accuracy = (cross_table.iloc[0, 0] + cross_table.iloc[1, 1]) / cross_table.loc['All', 'All'] if cross_table.shape == (3,3) else cross_table.iloc[1,0] / (cross_table.iloc[1, 0] + cross_table.iloc[0,0]) 
    precision = cross_table.iloc[1,1] / (cross_table.iloc[0,1] + cross_table.iloc[1,1]) if cross_table.shape == (3,3) else 0
    recall = cross_table.iloc[1,1] / (cross_table.iloc[1,0] + cross_table.iloc[1,1]) if cross_table.shape == (3,3) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    Accuracy.append(accuracy)
    Precision.append(precision)
    Recall.append(recall)
    F1_score.append(f1_score)
    Name.append(name)
    
  return(pd.DataFrame({"Model" : Name, "Accuracy" : Accuracy, "Precision" : Precision, "Recall" : Recall, "F1 Score" : F1_score}))

In [ ]:
get_metrics_df_3(df_with_text, to_check)

,Model,Accuracy,Precision,Recall,F1 Score
0,sentiment_score_vader,0.536842,0.422535,0.909091,0.576923
1,Vader_lexicon_scentences_sentiment,0.536842,0.415385,0.818182,0.551020
2,Vader_total_sentiments_based_on_words,0.463158,0.384615,0.909091,0.540541
3,climatebert/distilroberta-base-climate-sentiment,0.673684,0.517241,0.909091,0.659341
4,mrm8488/distilroberta-finetuned-financial-news...,0.400000,0.357143,0.909091,0.512821
